In [ ]:
from copy import deepcopy
from AI_Agent import AI
from Game import Kalaha
import matplotlib.pyplot as plt
import copy
import random
import numpy as np

In [ ]:
#Without pruning
depth = []
timings = []
for i in range(0, 9, 1):
    game = Kalaha()
    player=game.player1
    agent = AI(i)
    game_copy = deepcopy(game)
            
            
    start = time.time()
    
    best_move = agent.get_best_move(game_copy, game.get_state(), maximizing_player=False,pruning=False)

    end = time.time()
    
    depth.append(i)
    timings.append(round(end - start, 2))

In [ ]:
#with pruning
depth_ab = []
timings_ab = []
for i in range(0, 11, 1):
    game = Kalaha()
    player=game.player1
    agent = AI(i)
    game_copy = deepcopy(game)
            
            
    start = time.time()
    
    best_move = agent.get_best_move(game_copy, game.get_state(), maximizing_player=False,pruning=True)

    end = time.time()
    depth_ab.append(i)
    timings_ab.append(round(end - start, 2))

In [ ]:
plt.plot(depth, timings, label = "MinMax w/o pruning")
plt.plot(depth_ab, timings_ab, label = "MinMax w. pruning")

plt.xlabel('Search Depth')
plt.ylabel('Time [s]')
plt.legend(loc="upper left")
plt.show()
for i, timing in enumerate(timings): 
    print("Depth: {}  Time MinMax: {} s  Time MinMax AB Pruning {} s".format(i, timing, timings_ab[i]))